# Intro to Deep Learning Challenge 12

This is the response to the Coding challenge https://www.youtube.com/watch?v=t5qgjJIBy9g&t How to Make a Chatbot - Intro to Deep Learning #12

The code used here is referenced from https://github.com/llSourcell/How_to_make_a_chatbot

In [1]:
#Importing dependencies
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, merge
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re
from keras import layers

Using TensorFlow backend.


In [2]:

def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))


In [3]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
#match = layers.Dot([input_encoded_m, question_encoded], axes=(2, 2))
match = merge([input_encoded_m, question_encoded], mode='dot', dot_axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
#response = layers.add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = merge([match, input_encoded_c], mode='sum')
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
#answer = layers.concatenate([response, question_encoded])
answer = merge([response, question_encoded], mode='concat')

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          nb_epoch=120,
          validation_data=([inputs_test, queries_test], answers_test))

model.save_weights('memorynetwork.h5')

Extracting stories for the challenge: single_supporting_fact_10k


/home/eshwar/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 22)
answers_test shape: (1000, 22)
-
Compiling...
Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 5s - loss: 1.9621 - acc: 0.1604 - val_loss: 1.8074 - val_acc: 0.1540

In [4]:
#Evaluating model
model.evaluate([inputs_test, queries_test], answers_test)

 768/1000 [======================>.......] - ETA: 0s

[0.1086408862248063, 0.96499999999999997]

### Testing the model

In [5]:
#Testing for one of the test stories
test_stories[0]

(['John',
  'travelled',
  'to',
  'the',
  'hallway',
  '.',
  'Mary',
  'journeyed',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Where', 'is', 'John', '?'],
 'hallway')

In [7]:
#inputs_test, queries_test, answers_test contain the vectorized form of queries
answer = model.predict([inputs_test[0:1],queries_test[0:1]])

In [8]:
answer

array([[  2.66348875e-15,   2.95440518e-15,   2.98232847e-15,
          2.87560359e-15,   2.50128575e-15,   3.16328478e-15,
          2.73062776e-15,   2.84186330e-15,   2.75607834e-15,
          3.45590752e-06,   4.62986705e-10,   5.02218107e-08,
          9.99965310e-01,   2.94559371e-15,   3.31211079e-15,
          2.03716197e-10,   2.55002572e-15,   3.11823205e-05,
          2.84365266e-15,   3.36337069e-15,   2.75411323e-15,
          2.88562526e-15]], dtype=float32)

In [28]:
#Looking into structure of train_stories. It contains sentences in column 0 of each row, question in column 1 of each row 
#and answer in column 2 of each row
train_stories[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'went',
  'to',
  'the',
  'hallway',
  '.'],
 ['Where', 'is', 'Mary', '?'],
 'bathroom')

In [10]:
# Checking the structure of answers_train. We get model.predict() output in this form. We need to find out the index of  
# value which is close to one. Each index corresponds to a unique word like bathroom, hallway.
for i in range(0,5):
    print(answers_train[i],"\n")

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.] 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.] 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.] 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.] 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.] 



In [11]:
# Check out answers in train stories which corresponds to column 2 in each row as seen above
# Bathroom corresponds to index 9 as seen from results. Hallway corresponds to index 12 and so on.
for i in range(0,5):
    print(train_stories[i][2])

bathroom
hallway
hallway
office
bathroom


In [12]:
# Finding unique number of words as answers. Using sets to avoid duplicate. We have 6 unique answer words
answers = set()
for i in range(0,10000):
    answers.add(train_stories[i][2])
len(answers)

6

In [13]:
#Creating a dictionary to map answer_train vector to human readable word
def create_answer_dict():
    dict = {}
    for i in range (0,10000):
        for j in range(0,22):
            if(answers_train[i][j]==1):
                key = str(j)
                dict[key] = train_stories[i][2]
                if(len(dict) == 6): #Stopping at 6 as there are 6 unique answers
                    return dict
    return {}

In [14]:
answer_dictionary = create_answer_dict()

In [15]:
answer_dictionary #Gives words corresponding to index which are set to 1 in answers_train

{'10': 'bedroom',
 '11': 'garden',
 '12': 'hallway',
 '15': 'kitchen',
 '17': 'office',
 '9': 'bathroom'}

In [18]:
# Function to convert array returned by predict function to human readable string
def get_prediction(prediction_array):
    index_of_highest_value = prediction_array.argmax(axis = 1)
    #Return the string from dictionary for the index. Highest value will be closest to 1
    return answer_dictionary[str(index_of_highest_value[0])]

In [19]:
# Now to convert array from predict function to word
get_prediction(answer) #answer is the vector we got earlier as prediction

'hallway'

In [20]:
#Checking actual answer
answers_test[0:1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [22]:
get_prediction(answers_test[0:1]) #we get correct answer

'hallway'

### Checking the model for new story 

In [23]:
# Creating a new story using same words in dataset. Just performing permutations on locations and names to create new 
# story
my_story = [(['Daniel',
   'went',
   'back',
   'to',
   'the',
   'garden',
   '.',
   'Sandra',
   'moved',
   'to',
   'the',
   'hallway',
   '.',
   'John',
   'moved',
   'to',
   'the',
   'kitchen',
   '.',
   'Sandra',
   'journeyed',
   'to',
   'the',
   'bathroom',
   '.'],
  ['Where', 'is', 'Daniel', '?'],
  'garden'),]

In [24]:
#Generating vectors for my_story
story_vec, query_vec, answer_vec = vectorize_stories(my_story,word_idx,story_maxlen,query_maxlen)

In [25]:
# Using the model to predich
answer = model.predict([story_vec[0:1],query_vec[0:1]])

In [26]:
answer

array([[  4.24127862e-14,   4.09245019e-14,   3.88611127e-14,
          4.10845979e-14,   4.34425953e-14,   4.12122153e-14,
          3.99324433e-14,   4.36893902e-14,   4.37494313e-14,
          4.22524638e-04,   5.20839967e-05,   9.98867631e-01,
          2.24287930e-07,   4.03442437e-14,   4.35968231e-14,
          6.55144802e-04,   4.27528530e-14,   2.39628616e-06,
          4.12105618e-14,   4.38689205e-14,   4.31197368e-14,
          4.24472571e-14]], dtype=float32)

In [27]:
# Now convert array to word from function defined above
get_prediction(answer)

'garden'

We get the correct answer as garden. 